# Mouse-to-Human alignment

- This notebook creates the alignments between human and mouse that can then be used to check whether the concordant mutation in mouse can be modeled.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from Bio import SeqIO
import gzip
from Bio.Seq import Seq
import re
import gffutils

/Users/samgould/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
filepath = '/Volumes/Sam_G_SSD/2020-06-16-MSK-IMPACT_EDITED.txt'
impact_data = pd.read_csv(filepath, sep='\t')
impact_data

/Users/samgould/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (45,48,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,...,MOTIF_SCORE_CHANGE,PHENO,PICK,PUBMED,PolyPhen,SAS_MAF,SIFT,SOMATIC,SWISSPROT,SYMBOL,SYMBOL_SOURCE,TREMBL,TSL,Transcript,UNIPARC,VARIANT_CLASS,all_effects,amino_acid_change,cDNA_Change,cDNA_position,cdna_change,comments,n_depth,t_depth,transcript
0,BRCA2,675,MSKCC,GRCh37,13,32937315,32937315,+,splice_acceptor_variant,Splice_Site,SNP,G,G,C,rs81002874,NaN,P-0029279-T01-IM6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRCA2,0,MSKCC,37,13,32914437,32914438,+,NaN,NaN,DEL,GT,GT,G,rs80359550,NaN,P-0034227-T01-IM6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUTYH,4595,MSKCC,GRCh37,1,45798475,45798475,+,missense_variant,Missense_Mutation,SNP,T,T,C,rs34612342,NaN,P-0030735-T01-IM6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRCA2,675,MSKCC,GRCh37,13,32893302,32893302,+,frameshift_variant,Frame_Shift_Ins,INS,T,T,GCCGGGCGCGGTGG,NaN,NaN,P-0038798-T01-IM6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRCA1,0,MSKCC,37,17,41251824,41251825,+,NaN,NaN,DEL,TG,TG,T,rs80357872,NaN,P-0030162-T01-IM6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422817,SMARCA4,6597,MSKCC,GRCh37,19,11144132,11144132,+,missense_variant,Missense_Mutation,SNP,C,C,G,NaN,NaN,P-0052864-T01-XS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422818,BRAF,673,MSKCC,GRCh37,7,140453149,140453149,+,missense_variant,Missense_Mutation,SNP,C,C,G,rs121913361,NaN,P-0052867-T01-XS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422819,NRAS,4893,MSKCC,GRCh37,1,115258747,115258747,+,missense_variant,Missense_Mutation,SNP,C,C,T,rs121913237,NaN,P-0052951-T01-XS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422820,TERT,7015,MSKCC,GRCh37,5,1295521,1295521,+,upstream_gene_variant,5'Flank,SNP,A,A,T,NaN,NaN,P-0052951-T01-XS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#homology dataframe

homology_df = np.load('/Volumes/Sam_G_SSD/homology_table.npy', allow_pickle=True)
homology_df = pd.DataFrame(homology_df,columns=['gene','human gene name','mouse gene name','mouse id','mouse id version','mouse transcript'])

homology_df

,gene,human gene name,mouse gene name,mouse id,mouse id version,mouse transcript
0,ABL1,ABL1,Abl1,ENSMUSG00000026842,ENSMUSG00000026842.16,ENSMUST00000028190.12
1,AC004906.3,NONE,NONE,NONE,NONE,NONE
2,AC008738.1,NONE,NONE,NONE,NONE,NONE
3,ACTG1,ACTG1,Actg1,ENSMUSG00000062825,ENSMUSG00000062825.15,ENSMUST00000071555.12
4,ACVR1,ACVR1,Acvr1,ENSMUSG00000026836,ENSMUSG00000026836.15,ENSMUST00000056376.11
...,...,...,...,...,...,...
589,XRCC2,XRCC2,Xrcc2,ENSMUSG00000028933,ENSMUSG00000028933.11,ENSMUST00000030773.11
590,YAP1,YAP1,Yap1,ENSMUSG00000053110,ENSMUSG00000053110.13,ENSMUST00000086580.11
591,YES1,YES1,Yes1,ENSMUSG00000014932,ENSMUSG00000014932.15,ENSMUST00000168707.5
592,ZFHX3,ZFHX3,Zfhx3,ENSMUSG00000038872,ENSMUSG00000038872.10,ENSMUST00000043896.9


In [5]:
filename1 = '/Users/samgould/Desktop/FSR Lab/2022-03-17/gene_info.csv'
df1 = pd.read_csv(filename1)
df1

,gene,gene_id,transcript_id,chrom,gene_start,gene_end,transcript_start,transcript_end,strand
0,ABL1,ENSG00000097007.13,ENST00000318560.5,chr9,133589333,133763062,133710453,133763062,+
1,AC004906.3,ENSG00000237286.1,ENST00000423194.1,chr7,2983669,2986725,2983669,2986725,+
2,AC008738.1,ENSG00000230259.2,ENST00000425420.2,chr19,33790853,33793430,33790853,33793430,-
3,ACTG1,ENSG00000184009.5,ENST00000575842.1,chr17,79476997,79490873,79477015,79479807,-
4,ACVR1,ENSG00000115170.9,ENST00000263640.3,chr2,158592958,158732374,158592958,158731623,-
...,...,...,...,...,...,...,...,...,...
589,XRCC2,ENSG00000196584.2,ENST00000359321.1,chr7,152341864,152373250,152343589,152373250,-
590,YAP1,ENSG00000137693.9,ENST00000282441.5,chr11,101981192,102104154,101981192,102104154,+
591,YES1,ENSG00000176105.9,ENST00000314574.4,chr18,721588,812547,721748,812239,-
592,ZFHX3,ENSG00000140836.10,ENST00000268489.5,chr16,72816784,73093597,72816784,73082274,-


In [16]:
#loading in annotation databases for human and mouse
file = '/Volumes/Sam_G_SSD/gencode_v19.db'
db = gffutils.FeatureDB(file)

file_mouse = '/Volumes/Sam_G_SSD/GRCm38.p6 (mouse)/gencode_vM25.db'
db_mouse = gffutils.FeatureDB(file_mouse)

In [6]:
#loading in necessary genes for human
path = '/Volumes/Sam_G_SSD/human genome GrCh37 IMPACT genes/'
impact_genes = np.load(path + 'human_impact_genes_plusminus5000.npy', allow_pickle=True)
unique_genes = np.load(path + 'human_impact_genes_NAMES.npy', allow_pickle=True)

In [18]:
#loading in mouse genes
#loading in necessary genes for human
path = '/Volumes/Sam_G_SSD/mouse genome GRCm38.p6 IMPACT genes/'
mouse_genes = np.load(path + 'mouse_impact_genes_plusminus5000.npy', allow_pickle=True)
mouse_gene_names = np.load(path + 'mouse_impact_genes_NAMES.npy', allow_pickle=True)

# Generating human protein from transcript
- Also has ability to model mutations of interest in transcript

In [230]:
p53_idx = np.asarray(impact_data[impact_data['Hugo_Symbol']=='KRAS'].index)

In [231]:
idx = p53_idx[128]
impact_data.iloc[[idx]]

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,VARIANT_CLASS,all_effects,amino_acid_change,cDNA_Change,cDNA_position,cdna_change,comments,n_depth,t_depth,transcript
7764,KRAS,3845,MSKCC,GRCh37,12,25398281,25398281,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
#now doing an example of a gene on the minus strand

def human_protein(gene_name, unique_gene_list, tx_ids, SNP):
    tx_idx = unique_gene_list.index(gene_name)
    tx = tx_ids[tx_idx]

    #always order by + strand and then deal with - end genes later on
    cds = list(db.children(tx, order_by='+end', featuretype=['CDS'])) #only accessing cds
    #CDS refers to all of the exons in the gene

    #DEFINING the start and end of each coding exon
    start_end_cds = [[i.start, i.end] for i in cds]


    #23 = X
    seq_list = []
    for i in range(len(start_end_cds)):
        #need to put it in the correct indexing frame for the loaded in genes
        gene_start = df1[df1['gene']==gene_name]['gene_start'].values[0]
        #gene_end = df1[df1['gene']==gene_name]['gene_end'].values[0]

        start = start_end_cds[i][0]-1-gene_start+5000
        end = start_end_cds[i][1]-gene_start+5000
        sequence = impact_genes[tx_idx][start:end]

        seq_list.append(sequence)

    concatenated_seq = sum(seq_list, Seq(""))

    #take reverse complement to account for minus strand
    strand = df1[df1['gene']==gene_name]['strand'].values[0]
    if strand=='-': #if minus strand
        seq_true = concatenated_seq.reverse_complement()
    elif strand=='+':
        seq_true = concatenated_seq


    mRNA = seq_true.transcribe()
    prot_seq = mRNA.translate()

    #-_______________________________MUTATIONS_____________________________#
    #information about mutation of interest
    #FOR NOW ONLY LOOKING AT SNPs
    snp = SNP

    coding_location1 = impact_data.iloc[[idx]]['HGVSc'].values[0].split('.')[2]
    coding_location = int(re.findall(r'\d+', coding_location1)[0])

    seq_mut = seq_true[0:coding_location-1] + snp + seq_true[coding_location:]
    mRNA_mut = seq_mut.transcribe()
    prot_seq_mut = mRNA_mut.translate()

    
    return mRNA, prot_seq, mRNA_mut, prot_seq_mut

In [234]:
unique_gene_list = list(unique_genes)
tx_ids = np.asarray(df1['transcript_id'])

gene_name = 'KRAS'

SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1]
#need to expand to other mutation types

mRNA, prot_seq, mRNA_mut, prot_seq_mut = human_protein(gene_name, unique_gene_list, tx_ids, SNP)

In [235]:
#checking correctness
print(impact_data.iloc[[idx]]['HGVSp'].values[0])
idx_aa = int(re.findall(r'\d+', impact_data.iloc[[idx]]['HGVSp'].values[0])[0])
print(prot_seq[idx_aa-1])
print(prot_seq_mut[idx_aa-1])

p.Gly13Asp
G
D


# Generating mouse protein sequence from transcript
- Also has ability to model mutations of interest

In [237]:
homology_df

,gene,human gene name,mouse gene name,mouse id,mouse id version,mouse transcript
0,ABL1,ABL1,Abl1,ENSMUSG00000026842,ENSMUSG00000026842.16,ENSMUST00000028190.12
1,AC004906.3,NONE,NONE,NONE,NONE,NONE
2,AC008738.1,NONE,NONE,NONE,NONE,NONE
3,ACTG1,ACTG1,Actg1,ENSMUSG00000062825,ENSMUSG00000062825.15,ENSMUST00000071555.12
4,ACVR1,ACVR1,Acvr1,ENSMUSG00000026836,ENSMUSG00000026836.15,ENSMUST00000056376.11
...,...,...,...,...,...,...
589,XRCC2,XRCC2,Xrcc2,ENSMUSG00000028933,ENSMUSG00000028933.11,ENSMUST00000030773.11
590,YAP1,YAP1,Yap1,ENSMUSG00000053110,ENSMUSG00000053110.13,ENSMUST00000086580.11
591,YES1,YES1,Yes1,ENSMUSG00000014932,ENSMUSG00000014932.15,ENSMUST00000168707.5
592,ZFHX3,ZFHX3,Zfhx3,ENSMUSG00000038872,ENSMUSG00000038872.10,ENSMUST00000043896.9


In [238]:
def mouse_protein(human_gene_name, mouse_genes, db_mouse, unique_gene_list):
    
    tx_idx = unique_gene_list.index(human_gene_name)
    
    gene_id = homology_df[homology_df['gene']==human_gene_name]['mouse id version'].values[0]
    tx = homology_df[homology_df['gene']==human_gene_name]['mouse transcript'].values[0]


    #always order by + strand and then deal with - end genes later on
    cds = list(db_mouse.children(tx, order_by='+end', featuretype=['CDS'])) #only accessing cds
    #CDS refers to all of the exons in the gene

    #DEFINING the start and end of each coding exon
    start_end_cds = [[i.start, i.end] for i in cds]

    seq_list = []
    for i in range(len(start_end_cds)):
        #need to put it in the correct indexing frame for the loaded in genes
        #gene_start = df1[df1['gene']==gene_name]['gene_start'].values[0]
        #gene_end = df1[df1['gene']==gene_name]['gene_end'].values[0]

        gene_start = db_mouse[gene_id].start

        start = start_end_cds[i][0]-1-gene_start+5000
        end = start_end_cds[i][1]-gene_start+5000
        sequence = mouse_genes[tx_idx][start:end]

        seq_list.append(sequence)

    concatenated_seq = sum(seq_list, Seq(""))

    #take reverse complement to account for minus strand

    strand = db_mouse[tx].strand
    if strand=='-': #if minus strand
        seq_true = concatenated_seq.reverse_complement()
    elif strand=='+':
        seq_true = concatenated_seq


    mRNA = seq_true.transcribe()
    prot_seq = mRNA.translate()

    return mRNA, prot_seq

In [239]:
#now doing an example of a gene on the minus strand
unique_gene_list = list(unique_genes)
#tx_ids = np.asarray(df1['transcript_id'])
human_gene_name = 'KRAS'

mRNA, protein = mouse_protein(human_gene_name, mouse_genes, db_mouse, unique_gene_list)

# Protein Sequence alignment
- pairwise global alignment
    - match score = 1
    - gap penalty = 0

In [243]:
from Bio import AlignIO
import Bio.Align
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [242]:
#An example using P53
#Generating mouse protein
unique_gene_list = list(unique_genes)
human_gene_name = 'TP53'

mRNA, mouse_p53 = mouse_protein(human_gene_name, mouse_genes, db_mouse, unique_gene_list)

#Generating human protein
tx_ids = np.asarray(df1['transcript_id'])

gene_name = 'TP53'

SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1] #this is just a filler; don't care about it; originally meant for KRAS
#need to expand to other mutation types

mRNA, human_p53, mRNA_mut, prot_seq_mut = human_protein(gene_name, unique_gene_list, tx_ids, SNP)

In [247]:
alignments = pairwise2.align.globalxx(human_p53, mouse_p53)
print(format_alignment(*alignments[0]))

M---EEP-QSDP-SV-EP-PLSQETFSD-LWKLLPENNVLSP---LPSQA---MDDLMLS-PD-DI-EQW-FT-EDPGPD-EAP-RMPEAAPPVAP--APAA--P-T--PA---APAPA-PSWPLSSS-VPSQKTYQGS-YGFR-LGFLH-SGTAKSVT-CTYSPA-LNKM-FCQLAKTCPVQLWVDS-TPPP-GT-RVRAMAIYKQ-SQHMTEVVRRCPHHERCSDS-DGLAPPQHLIRVEGNLRV--EYLD-DRN-TFRHSVVVPYEPPEV-GSDC--TTIHYN-YMCNSSCMGGMNRRPILTIITLEDSSGNLLGRN-SFEVRVCACPGRDRRTEEENL-RKKGE---PHHELPPGST-KRALPNNTS--S-SPQP-KKKPLDGEYFTLQ-IRGRE-RFEMFRELNEALELKDAQ-AGK-EPG--G-SRAHSSH-LKS-KKGQSTSRHKKL-MF-KTE--GPDSD
|   ||  |||  |  |  ||||||||  ||||||      |   |||     |||| |  |  |  |   |  |  ||  ||  |       |    ||||  | |  |    ||||| | ||| || |||||||||  |||  ||||  |||||||  |||||  |||  |||||||||||||| | | || |  |||||||||  |||||||||||||||||||  ||||||||||||||||    |||  ||  ||||||||||||||  ||    |||||  ||||||||||||||||||||||||||||||||  ||||||||||||||||||||  ||| |   |  ||||||  |||||  |   | || | ||||||||||||  ||||  |||||||||||||||||  |   |    | ||||||  ||  |||||||||||  |  |    |||||
MTAMEE-SQSD-IS-LE-LPLSQETFS-GLWKLLP------PEDILPS--PHCMDDL-L-LP

In [249]:
#producing indeces of homologous sequences in each
def align_idx(human_prot, mouse_prot):
    "Returns indexes of HOMOLOGOUS regions in mouse and human"
    
    alignments = pairwise2.align.globalxx(human_prot, mouse_prot)

    match_idx = match_index(*alignments[0])
    
    human_aln_seq = alignments[0][0] #human alignment sequence
    mouse_aln_seq = alignments[0][1] #human alignment sequence

    human_idx = []
    mouse_idx = []
    
    for index in match_idx:
        #calculate human original idx
        gaps = human_aln_seq[:index].count('-')
        original_index = len(human_aln_seq[:index]) - gaps
        human_idx.append(original_index)
        
        #calculate mouse original idx
        gaps = mouse_aln_seq[:index].count('-')
        original_index = len(mouse_aln_seq[:index]) - gaps
        mouse_idx.append(original_index)

    return human_idx, mouse_idx
    

## Now performing this alignment for each of the orthologous proteins and saving to arrays

- important note: ignore non-coding transcripts -- this is specific to MAP3K14 (of which only 2 muts are recorded)

In [152]:
homology_df

,gene,human gene name,mouse gene name,mouse id,mouse id version,mouse transcript
0,ABL1,ABL1,Abl1,ENSMUSG00000026842,ENSMUSG00000026842.16,ENSMUST00000028190.12
1,AC004906.3,NONE,NONE,NONE,NONE,NONE
2,AC008738.1,NONE,NONE,NONE,NONE,NONE
3,ACTG1,ACTG1,Actg1,ENSMUSG00000062825,ENSMUSG00000062825.15,ENSMUST00000071555.12
4,ACVR1,ACVR1,Acvr1,ENSMUSG00000026836,ENSMUSG00000026836.15,ENSMUST00000056376.11
...,...,...,...,...,...,...
589,XRCC2,XRCC2,Xrcc2,ENSMUSG00000028933,ENSMUSG00000028933.11,ENSMUST00000030773.11
590,YAP1,YAP1,Yap1,ENSMUSG00000053110,ENSMUSG00000053110.13,ENSMUST00000086580.11
591,YES1,YES1,Yes1,ENSMUSG00000014932,ENSMUSG00000014932.15,ENSMUST00000168707.5
592,ZFHX3,ZFHX3,Zfhx3,ENSMUSG00000038872,ENSMUSG00000038872.10,ENSMUST00000043896.9


In [385]:
#Generating human and mouse protein alignments and indeces
unique_gene_list = list(unique_genes)
tx_ids = np.asarray(df1['transcript_id'])
SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1] 
#this is just a filler; don't care about it; originally a mutation meant for KRAS

human_alignments = []
mouse_alignments = []

#iterate through the genes
for human_gene_name in unique_gene_list:
    
    tx_idx = unique_gene_list.index(human_gene_name)
    tx = tx_ids[tx_idx]
    cds = list(db.children(tx, order_by='+end', featuretype=['CDS'])) #only accessing cds    
    
    
    #need to check if there exists an orthologue in mouse
    #if not, skip the alignment process
    if homology_df[homology_df['gene']==human_gene_name]['mouse transcript'].values[0] == 'NONE':
        human_alignments.append(['no_ortholog'])
        mouse_alignments.append(['no_ortholog'])
    
    elif len(cds)==0: #ignore non-coding transcripts -- this is specific to MAP3K14 (of which only 2 muts are recorded)
        
        human_alignments.append(['non_coding'])
        mouse_alignments.append(['non_coding'])
        
    else:

        gene_name = human_gene_name

        mRNA, mouse_prot = mouse_protein(gene_name, mouse_genes, db_mouse, unique_gene_list)

        mRNA, human_prot, mRNA_mut, prot_seq_mut = human_protein(gene_name, unique_gene_list, tx_ids, SNP)

        human_idx, mouse_idx = align_idx(human_prot, mouse_prot)
        
        human_alignments.append(human_idx)
        mouse_alignments.append(mouse_idx)
        

/Users/samgould/anaconda3/lib/python3.7/site-packages/Bio/Seq.py:2715: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [386]:
#and now saving these results

human_align_array = np.asarray(human_alignments, dtype=object)
mouse_align_array = np.asarray(mouse_alignments, dtype=object)

In [387]:
path = '/Volumes/Sam_G_SSD/human_mouse_alignments/'
#np.save(path+'human_alignment_idx.npy', human_align_array)
#np.save(path+'mouse_alignment_idx.npy', mouse_align_array)

In [270]:
#For loading these alignment files back in
path = '/Volumes/Sam_G_SSD/human_mouse_alignments/'
human_prot_align = np.load(path+'human_alignment_idx.npy', allow_pickle=True)
mouse_prot_align = np.load(path+'mouse_alignment_idx.npy', allow_pickle=True)



## Generating list of DNA sequence positions of each codon in given coding sequence
- This allows mapping of the mutations onto protein sequence
- Need to do this for both mouse AND human sequences

- Dealing with the plus and minus strands could be a bit annoying here. In general the mutations in the IMPACT dataset are reported as the plus strand, so I will follow this convention. But, this will need to be double checked for sure.

In [369]:
impact_data['Strand'].unique()
#all of the mutations are recorded with respect to the + strand in the IMPACT dataset

array(['+'], dtype=object)

In [298]:
#now doing an example of a gene on the minus strand

def human_protein_2(gene_name, unique_gene_list, tx_ids, SNP):
    tx_idx = unique_gene_list.index(gene_name)
    tx = tx_ids[tx_idx]

    #always order by + strand and then deal with - end genes later on
    cds = list(db.children(tx, order_by='+end', featuretype=['CDS'])) #only accessing cds
    #CDS refers to all of the exons in the gene

    #DEFINING the start and end of each coding exon
    start_end_cds = [[i.start, i.end] for i in cds]


    #23 = X
    seq_list = []
    for i in range(len(start_end_cds)):
        #need to put it in the correct indexing frame for the loaded in genes
        gene_start = df1[df1['gene']==gene_name]['gene_start'].values[0]
        #gene_end = df1[df1['gene']==gene_name]['gene_end'].values[0]

        start = start_end_cds[i][0]-1-gene_start+5000
        end = start_end_cds[i][1]-gene_start+5000
        sequence = impact_genes[tx_idx][start:end]

        seq_list.append(sequence)

    concatenated_seq = sum(seq_list, Seq(""))

    #take reverse complement to account for minus strand
    strand = df1[df1['gene']==gene_name]['strand'].values[0]
    if strand=='-': #if minus strand
        seq_true = concatenated_seq.reverse_complement()
    elif strand=='+':
        seq_true = concatenated_seq


    mRNA = seq_true.transcribe()
    prot_seq = mRNA.translate()

    #-_______________________________MUTATIONS_____________________________#
    #information about mutation of interest
    #FOR NOW ONLY LOOKING AT SNPs
    snp = SNP

    coding_location1 = impact_data.iloc[[idx]]['HGVSc'].values[0].split('.')[2]
    coding_location = int(re.findall(r'\d+', coding_location1)[0])

    seq_mut = seq_true[0:coding_location-1] + snp + seq_true[coding_location:]
    mRNA_mut = seq_mut.transcribe()
    prot_seq_mut = mRNA_mut.translate()

    
    return mRNA, prot_seq, mRNA_mut, prot_seq_mut, start_end_cds, concatenated_seq

In [299]:
def mouse_protein_2(human_gene_name, mouse_genes, db_mouse, unique_gene_list):
    
    tx_idx = unique_gene_list.index(human_gene_name)
    
    gene_id = homology_df[homology_df['gene']==human_gene_name]['mouse id version'].values[0]
    tx = homology_df[homology_df['gene']==human_gene_name]['mouse transcript'].values[0]


    #always order by + strand and then deal with - end genes later on
    cds = list(db_mouse.children(tx, order_by='+end', featuretype=['CDS'])) #only accessing cds
    #CDS refers to all of the exons in the gene

    #DEFINING the start and end of each coding exon
    start_end_cds = [[i.start, i.end] for i in cds]

    seq_list = []
    for i in range(len(start_end_cds)):
        #need to put it in the correct indexing frame for the loaded in genes
        #gene_start = df1[df1['gene']==gene_name]['gene_start'].values[0]
        #gene_end = df1[df1['gene']==gene_name]['gene_end'].values[0]

        gene_start = db_mouse[gene_id].start

        start = start_end_cds[i][0]-1-gene_start+5000
        end = start_end_cds[i][1]-gene_start+5000
        sequence = mouse_genes[tx_idx][start:end]

        seq_list.append(sequence)

    concatenated_seq = sum(seq_list, Seq(""))

    #take reverse complement to account for minus strand

    strand = db_mouse[tx].strand
    if strand=='-': #if minus strand
        seq_true = concatenated_seq.reverse_complement()
    elif strand=='+':
        seq_true = concatenated_seq


    mRNA = seq_true.transcribe()
    prot_seq = mRNA.translate()

    return mRNA, prot_seq, start_end_cds, concatenated_seq

In [300]:
df1[df1['gene']=='ABL1']

,gene,gene_id,transcript_id,chrom,gene_start,gene_end,transcript_start,transcript_end,strand
0,ABL1,ENSG00000097007.13,ENST00000318560.5,chr9,133589333,133763062,133710453,133763062,+


In [413]:
unique_gene_list = list(unique_genes)
tx_ids = np.asarray(df1['transcript_id'])
SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1] 

gene_name = 'TP53'

mRNA, mouse_prot, start_end_cds_m, concatenated_seq_m = mouse_protein_2(gene_name, mouse_genes, db_mouse, unique_gene_list)

mRNA, human_prot, mRNA_mut, prot_seq_mut, start_end_cds_h, concatenated_seq_h = human_protein_2(gene_name, unique_gene_list, tx_ids, SNP)




In [414]:
gene_name = 'TP53'
gene_id = homology_df[homology_df['gene']==gene_name]['mouse id version'].values[0]
strand = db_mouse[gene_id].strand
strand

'+'

In [430]:
def mouse_codons(gene_name, start_end_cds_m, concatenated_seq_m):
    #and now doing the same, but accounting for plus or minus strand
    gene_id = homology_df[homology_df['gene']==gene_name]['mouse id version'].values[0]
    strand = db_mouse[gene_id].strand

    if strand=='+':
        #generating list with fleshed out indeces
        full_list = []
        for i in start_end_cds_m:
            k = list(range(i[0], i[1]+1))
            for num in k:
                full_list.append(num)

        #and splitting it up by 3s to create codons
        codons = []
        for i in range(0, len(full_list), 3):
            codons.append(full_list[i:i+3])


        #now checking and generating corresponding sequence
        a_a_list = []
        dna_seq_list = []
        for codon_num in range(len(codons)):

            tx_idx = unique_gene_list.index(gene_name)
            gene_id = homology_df[homology_df['gene']==gene_name]['mouse id version'].values[0]

            gene_start = db_mouse[gene_id].start

            c1 = codons[codon_num][0]-1-gene_start+5000
            c2 = codons[codon_num][1]-1-gene_start+5000
            c3 = codons[codon_num][2]-1-gene_start+5000

            seq_list = []
            seq_list.append(mouse_genes[tx_idx][c1])
            seq_list.append(mouse_genes[tx_idx][c2])
            seq_list.append(mouse_genes[tx_idx][c3])
            concatenated_seq = sum(seq_list, Seq(""))
            mrna = concatenated_seq.transcribe()
            aa = mrna.translate()


            a_a_list.append(aa)
            dna_seq_list.append(concatenated_seq) #contains dna sequence corresponding

    elif strand=='-':
        #generating list with fleshed out indeces
        full_list = []
        for i in start_end_cds_m:
            k = list(range(i[0], i[1]+1))
            for num in k:
                full_list.append(num)


        #reversing to account for - strand
        full_list = full_list[::-1]

        #and splitting it up by 3s to create codons
        codons = []
        for i in range(0, len(full_list), 3):
            codons.append(full_list[i:i+3])


        #now checking and generating corresponding sequence
        a_a_list = []
        dna_seq_list = []
        for codon_num in range(len(codons)):

            tx_idx = unique_gene_list.index(gene_name)
            gene_id = homology_df[homology_df['gene']==gene_name]['mouse id version'].values[0]

            gene_start = db_mouse[gene_id].start

            c1 = codons[codon_num][0]-1-gene_start+5000
            c2 = codons[codon_num][1]-1-gene_start+5000
            c3 = codons[codon_num][2]-1-gene_start+5000

            seq_list = []
            seq_list.append(mouse_genes[tx_idx][c1])
            seq_list.append(mouse_genes[tx_idx][c2])
            seq_list.append(mouse_genes[tx_idx][c3])
            concatenated_seq = sum(seq_list, Seq(""))
            mrna = concatenated_seq.complement().transcribe()
            aa = mrna.translate()

            a_a_list.append(aa)
            dna_seq_list.append(concatenated_seq) #contains dna sequence corresponding

    return codons, dna_seq_list, a_a_list

In [436]:
unique_gene_list = list(unique_genes)
tx_ids = np.asarray(df1['transcript_id'])
SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1] 

gene_name = 'ABL1'

mRNA, mouse_prot, start_end_cds_m, concatenated_seq_m = mouse_protein_2(gene_name, mouse_genes, db_mouse, unique_gene_list)

codons, dna_seq_list, a_a_list = mouse_codons(gene_name, start_end_cds_m, concatenated_seq_m)

In [437]:
for i in range(len(mouse_prot)):
    if mouse_prot[i] == a_a_list[i][0]:
        continue
    else:
        print(i)

In [439]:
gene_id = homology_df[homology_df['gene']==gene_name]['mouse id version'].values[0]
strand = db_mouse[gene_id].strand

print(strand)
print(mouse_prot[57])
print(a_a_list[57][0])
print(dna_seq_list[57])

+
P
P
CCC


In [443]:
def human_codons(gene_name, start_end_cds_h, concatenated_seq_h): 
    #other variables defined within that are not inputs, but simply defined in previous cells
    
    tx_idx = unique_gene_list.index(gene_name)
    tx = tx_ids[tx_idx]
    
    strand = df1[df1['gene']==gene_name]['strand'].values[0]

    
    if strand=='+':
        #generating list with fleshed out indeces
        full_list = []
        for i in start_end_cds_h:
            k = list(range(i[0], i[1]+1))
            for num in k:
                full_list.append(num)

        #and splitting it up by 3s to create codons
        codons = []
        for i in range(0, len(full_list), 3):
            codons.append(full_list[i:i+3])


        #now checking and generating corresponding sequence
        a_a_list = []
        dna_seq_list = []
        for codon_num in range(len(codons)):

            gene_start = df1[df1['gene']==gene_name]['gene_start'].values[0]

            c1 = codons[codon_num][0]-1-gene_start+5000
            c2 = codons[codon_num][1]-1-gene_start+5000
            c3 = codons[codon_num][2]-1-gene_start+5000

            seq_list = []
            seq_list.append(impact_genes[tx_idx][c1])
            seq_list.append(impact_genes[tx_idx][c2])
            seq_list.append(impact_genes[tx_idx][c3])
            concatenated_seq = sum(seq_list, Seq(""))
            mrna = concatenated_seq.transcribe()
            aa = mrna.translate()


            a_a_list.append(aa)
            dna_seq_list.append(concatenated_seq) #contains dna sequence corresponding

    elif strand=='-':
        #generating list with fleshed out indeces
        full_list = []
        for i in start_end_cds_h:
            k = list(range(i[0], i[1]+1))
            for num in k:
                full_list.append(num)


        #reversing to account for - strand
        full_list = full_list[::-1]

        #and splitting it up by 3s to create codons
        codons = []
        for i in range(0, len(full_list), 3):
            codons.append(full_list[i:i+3])


        #now checking and generating corresponding sequence
        a_a_list = []
        dna_seq_list = []
        for codon_num in range(len(codons)):

            gene_start = df1[df1['gene']==gene_name]['gene_start'].values[0]

            c1 = codons[codon_num][0]-1-gene_start+5000
            c2 = codons[codon_num][1]-1-gene_start+5000
            c3 = codons[codon_num][2]-1-gene_start+5000

            seq_list = []
            seq_list.append(impact_genes[tx_idx][c1])
            seq_list.append(impact_genes[tx_idx][c2])
            seq_list.append(impact_genes[tx_idx][c3])
            concatenated_seq = sum(seq_list, Seq(""))
            
            mrna = concatenated_seq.complement().transcribe()
            aa = mrna.translate()

            a_a_list.append(aa)
            dna_seq_list.append(concatenated_seq) #contains dna sequence corresponding

    return codons, dna_seq_list, a_a_list


In [480]:
unique_gene_list = list(unique_genes)
tx_ids = np.asarray(df1['transcript_id'])
SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1] 

gene_name = 'ABL1'

#mRNA, mouse_prot, start_end_cds_m, concatenated_seq_m = mouse_protein_2(gene_name, mouse_genes, db_mouse, unique_gene_list)

mRNA, human_prot, mRNA_mut, prot_seq_mut, start_end_cds_h, concatenated_seq_h = human_protein_2(gene_name, unique_gene_list, tx_ids, SNP)


codons, dna_seq_list, a_a_list = human_codons(gene_name, start_end_cds_h, concatenated_seq_h)

In [481]:
print(human_prot)

MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSST

In [450]:
for i in range(len(human_prot)):
    if human_prot[i] == a_a_list[i][0]:
        continue
    else:
        print(i)

In [451]:

strand = df1[df1['gene']==gene_name]['strand'].values[0]

print(strand)
print(human_prot[57])
print(a_a_list[57][0])
print(dna_seq_list[57])

-
P
P
GGT


## Now iterating through and generating this DNA sequence-codon match for each gene

- excluding same genes as before
- Again, important to be mindful of strand. Everything I've written here looks at + strand for sequence
- Had to exclude RYBP because of weird error with annotation file
    - This means that the alignment with mouse generated above is also fucked up
    - I can go back and fix this manually...
    - I should get a list and count of all of the excluded genes...

In [467]:
#Generating human and mouse protein alignments and indeces

tx_ids = np.asarray(df1['transcript_id'])
SNP = impact_data.iloc[[idx]]['HGVSc'].values[0][-1] 
#this is just a filler; don't care about it; originally a mutation meant for KRAS

human_codon_locations = []
human_codon_seqs = []
human_aa = []

mouse_codon_locations = []
mouse_codon_seqs = []
mouse_aa = []


#iterate through the genes
for human_gene_name in unique_gene_list:
    
    tx_idx = unique_gene_list.index(human_gene_name)
    tx = tx_ids[tx_idx]
    cds = list(db.children(tx, order_by='+end', featuretype=['CDS'])) #only accessing cds    
    
    
    #need to check if there exists an orthologue in mouse
    #if not, skip the alignment process
    if homology_df[homology_df['gene']==human_gene_name]['mouse transcript'].values[0] == 'NONE':
        
        human_codon_locations.append(['no_ortholog'])
        human_codon_seqs.append(['no_ortholog'])
        human_aa.append(['no_ortholog'])

        mouse_codon_locations.append(['no_ortholog'])
        mouse_codon_seqs.append(['no_ortholog'])
        mouse_aa.append(['no_ortholog'])
        
        
    elif len(cds)==0: #ignore non-coding transcripts -- this is specific to MAP3K14 (of which only 2 muts are recorded)
        
        human_codon_locations.append(['non_coding'])
        human_codon_seqs.append(['non_coding'])
        human_aa.append(['non_coding'])

        mouse_codon_locations.append(['non_coding'])
        mouse_codon_seqs.append(['non_coding'])
        mouse_aa.append(['non_coding'])
        
        
    elif human_gene_name == unique_gene_list[480]: #avoiding weird RYBP error

        human_codon_locations.append(['annotation_error'])
        human_codon_seqs.append(['annotation_error'])
        human_aa.append(['annotation_error'])

        mouse_codon_locations.append(['annotation_error'])
        mouse_codon_seqs.append(['annotation_error'])
        mouse_aa.append(['annotation_error'])

    else:

        gene_name = human_gene_name

        #first human
        mRNA, human_prot, mRNA_mut, prot_seq_mut, start_end_cds_h, concatenated_seq_h = human_protein_2(gene_name, unique_gene_list, tx_ids, SNP)
        codons, dna_seq_list, a_a_list = human_codons(gene_name, start_end_cds_h, concatenated_seq_h)
        
        human_codon_locations.append(codons)
        human_codon_seqs.append(dna_seq_list)
        human_aa.append(a_a_list)
        
        #and then mouse
        
        mRNA, mouse_prot, start_end_cds_m, concatenated_seq_m = mouse_protein_2(gene_name, mouse_genes, db_mouse, unique_gene_list)
        codons, dna_seq_list, a_a_list = mouse_codons(gene_name, start_end_cds_m, concatenated_seq_m)
        
        mouse_codon_locations.append(codons)
        mouse_codon_seqs.append(dna_seq_list)
        mouse_aa.append(a_a_list)
        
        

In [468]:
len(mouse_aa)

594

In [471]:
#and saving the resulting arrays etc.
path = '/Volumes/Sam_G_SSD/human_mouse_alignments/'

#human_codon_locations_arr = np.asarray(human_codon_locations, dtype=object)
#human_codon_seqs_arr = np.asarray(human_codon_seqs, dtype=object)
#human_aa_arr = np.asarray(human_aa, dtype=object)

#mouse_codon_locations_arr = np.asarray(mouse_codon_locations, dtype=object)
#mouse_codon_seqs_arr = np.asarray(mouse_codon_seqs, dtype=object)
#mouse_aa_arr = np.asarray(mouse_aa, dtype=object)

#np.save(path+'human_codon_locations.npy', human_codon_locations_arr)
#np.save(path+'human_codon_seqs.npy', human_codon_seqs_arr)
#np.save(path+'human_aa.npy', human_aa_arr)

#np.save(path+'mouse_codon_locations.npy', mouse_codon_locations_arr)
#np.save(path+'mouse_codon_seqs.npy', mouse_codon_seqs_arr)
#np.save(path+'mouse_aa.npy', mouse_aa_arr)

In [10]:
#and for loading in this information
path = '/Volumes/Sam_G_SSD/human_mouse_alignments/'

human_codon_locations = np.load(path+'human_codon_locations.npy', allow_pickle=True)
human_codon_seqs = np.load(path+'human_codon_seqs.npy', allow_pickle=True)
human_aa = np.load(path+'human_aa.npy', allow_pickle=True)

mouse_codon_locations = np.load(path+'mouse_codon_locations.npy', allow_pickle=True)
mouse_codon_seqs = np.load(path+'mouse_codon_seqs.npy', allow_pickle=True)
mouse_aa = np.load(path+'mouse_aa.npy', allow_pickle=True)

## Enumerating all of the genes that are excluded from analysis

In [17]:
excluded_genes = []
for i in range(len(mouse_aa)):
    
    if len(mouse_aa[i])==1:
        excluded_genes.append(unique_genes[i])

In [20]:
print('There are ' + str(len(excluded_genes)) + ' excluded genes.')

There are 56 excluded genes.


In [29]:
num_mutations_excluded = 0
num_excluded = []
for i in excluded_genes:
    num_exc = len(impact_data[impact_data['Hugo_Symbol']==i])
    num_mutations_excluded += num_exc
    num_excluded.append(num_exc)

In [30]:
print('There are ' + str(num_mutations_excluded) + ' mutations excluded.')

percent = num_mutations_excluded/len(impact_data)
print('This represents ' + str(np.round(percent*100, 2)) + '% of all mutations in the dataset.')


There are 7762 mutations excluded.
This represents 1.84% of all mutations in the dataset.


In [33]:
excluded = pd.DataFrame(excluded_genes, columns=['gene'])
excluded['num mutations']= num_excluded
excluded

,gene,num mutations
0,AC004906.3,1
1,AC008738.1,6
2,AP003419.16,1
3,C1orf147,4
4,CD58,80
5,CTD-2278I10.6,1
6,CTD-2330K9.2,1
7,CTD-2540B15.7,1
8,CTD-2561B21.3,1
9,FAM175A,192


## Example: mapping from sequence to codon

In [27]:
i=0
codon_locs = human_codon_locations[i]
codon_seqs = human_codon_seqs[i]
aa = human_aa[i]


ind = np.where(np.array(codon_locs) == 133710839)
ind

(array([1]), array([2]))

In [28]:
#located in 2nd codon (index=1) and 3rd place in codon (index=2)
codon_num = ind[0][0]
seq_index_in_codon = ind[1][0]#ranges from 0,1,or 2
print(codon_seqs[codon_num])
print(codon_seqs[codon_num][seq_index_in_codon])
print(aa[codon_num])



TTG
G
L


In [34]:
impact_data.iloc[[0]]

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,VARIANT_CLASS,all_effects,amino_acid_change,cDNA_Change,cDNA_position,cdna_change,comments,n_depth,t_depth,transcript
0,BRCA2,675,MSKCC,GRCh37,13,32937315,32937315,+,splice_acceptor_variant,Splice_Site,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRCA2,0,MSKCC,37,13,32914437,32914438,+,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUTYH,4595,MSKCC,GRCh37,1,45798475,45798475,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRCA2,675,MSKCC,GRCh37,13,32893302,32893302,+,frameshift_variant,Frame_Shift_Ins,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRCA1,0,MSKCC,37,17,41251824,41251825,+,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422817,SMARCA4,6597,MSKCC,GRCh37,19,11144132,11144132,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422818,BRAF,673,MSKCC,GRCh37,7,140453149,140453149,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422819,NRAS,4893,MSKCC,GRCh37,1,115258747,115258747,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422820,TERT,7015,MSKCC,GRCh37,5,1295521,1295521,+,upstream_gene_variant,5'Flank,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
